In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/brain/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/brain/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/brain/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
sns.heatmap([])

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [11]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 467s 6s/step - loss: 0.4918 - accuracy: 0.7792 - val_loss: 0.3260 - val_accuracy: 0.8517
Epoch 2/5
75/75 [==============================] - 310s 4s/step - loss: 0.2112 - accuracy: 0.9208 - val_loss: 0.1536 - val_accuracy: 0.9450
Epoch 3/5
75/75 [==============================] - 309s 4s/step - loss: 0.0772 - accuracy: 0.9817 - val_loss: 0.1056 - val_accuracy: 0.9583
Epoch 4/5
75/75 [==============================] - 291s 4s/step - loss: 0.0617 - accuracy: 0.9837 - val_loss: 0.0234 - val_accuracy: 0.9900
Epoch 5/5
75/75 [==============================] - 293s 4s/step - loss: 0.0249 - accuracy: 0.9933 - val_loss: 0.0214 - val_accuracy: 0.9883


In [14]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
model = load_model('/content/Model.h5')
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [16]:
image = Image.open("/content/drive/MyDrive/brain/Brain_Tumor_Detection/test/pred37.jpg").convert("RGB")
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
data[0] = normalized_image_array

In [17]:
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)

1/1 [==============================] - 0s 182ms/step
Result:  You have Brain Tumor

Accuracy:  0.9987558
